In [10]:
import os
import paramiko
import requests

In [2]:
def check_remote_file_exists(host, port, username, password, remote_path):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        # Connect to the remote server
        client.connect(host, port, username, password)

        # Command to check if the file or folder exists
        command = f'test -e "{remote_path}" && echo Exists || echo Does not exist'
        stdin, stdout, stderr = client.exec_command(command)

        # Read the output
        result = stdout.read().decode().strip()
        error = stderr.read().decode().strip()

        if error:
            raise Exception(f"Error checking file existence: {error}")

        return result == "Exists"
    finally:
        client.close()

In [3]:
# create the folder
def create_folder(host, username, password, remote_path):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # Connect to the server
        client.connect(host, username=username, password=password)

        # Open SFTP session
        sftp = client.open_sftp()

        # Create the remote directory
        sftp.mkdir(remote_path)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the SFTP session and SSH connection
        if sftp:
            sftp.close()
        client.close()

In [4]:
def upload_file_to_server(host, port, username, password, local_filepath, remote_filepath):
    # Initialize the SSH client
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # Connect to the server
        ssh_client.connect(hostname=host, port=port, username=username, password=password)

        # Initialize SFTP session
        with ssh_client.open_sftp() as sftp:
            # Upload file
            sftp.put(local_filepath, remote_filepath)
            print(f"File uploaded successfully to {remote_filepath}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        ssh_client.close()

In [5]:
# upload the bash data
def write_bash_file(sample):
    bash = [
        "#!/bin/bash",
        "#SBATCH --job-name={}    ### Job Name".format(sample),
        "#SBATCH --output={}.out       ### File in which to store job output".format(sample),
        "#SBATCH --error={}.err        ### File in which to store job error messages".format(sample),
        "#SBATCH --qos=normal",
        "#SBATCH --time=1-00:00:00     ### Wall clock time limit in Days-HH:MM:SS",
        "#SBATCH --nodes=1             ### Node count required for the job",
        "#SBATCH --ntasks-per-node=20   ### Number of tasks to be launched per Node",
        "#SBATCH --partition=centos7",
        "#SBATCH --mem=128000",
                            
        "module load anaconda3/2023.07",
        "export CONDA_ENVS_PATH=/lustre/project/hdeng2/ygong/env",
        "source activate st",
        
        "python segmentation.py"
    ]
    with open("segmentation.sh", 'w') as file:
        for line in bash:
            file.write(line + "\n")

In [6]:
# write the file on the cypress
host = "cypress1.tulane.edu"
username = "ygong"
password = "Tul@ne2021"
port = 22

# check the folder exist
dir_exist = check_remote_file_exists(
    host = host, 
    username = username, 
    password = password, 
    port = port, 
    remote_path = "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation"
)

# create the folder if not exist
if not dir_exist:
    create_folder(
        host, 
        username, 
        password, 
        "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation"
    )

In [20]:
for file in os.listdir("../../processed_data/"):
    dir_exist = check_remote_file_exists(
        host = host, 
        username = username, 
        password = password, 
        port = port, 
        remote_path = "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}".format(file)
    )

    if not dir_exist:
        create_folder(
            host, 
            username, 
            password, 
            "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}".format(file)
        )
        
    # Create Result folder for each sample
    dir_exist = check_remote_file_exists(
        host = host, 
        username = username, 
        password = password, 
        port = port, 
        remote_path = "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}/Result".format(file)
    )

    if not dir_exist:
        create_folder(
            host, 
            username, 
            password, 
            "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}/Result".format(file)
        )

In [7]:
for sample in os.listdir("../../processed_data/"):
    with open("segmentation.py", 'w') as file:
        with open("cell_segmentation_test.py", "r") as file1:
            for line_number, line in enumerate(file1, start=1):
                file.write(line)
            for content in ["if __name__ == '__main__':", "    cell_segmentation('{}')".format(sample)]:
                file.write(content + "\n")
    upload_file_to_server(
        host = host,
        port=22,
        username = username, 
        password = password, 
        local_filepath= "segmentation.py", 
        remote_filepath = "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}/segmentation.py".format(sample)
    )

File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/C02248B5/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/A02092E1/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01806B5/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01809A3/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B02009F6/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01806B6/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01809A4/segmentation.py
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/D02175A6/segmentation.py
File uploaded successfully to /home/ygon

In [89]:
for sample in os.listdir("../../processed_data/"):
    write_bash_file(sample)
    upload_file_to_server(
        host = host,
        port=22,
        username = username, 
        password = password, 
        local_filepath= "segmentation.sh", 
        remote_filepath = "/home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/{}/segmentation.sh".format(sample)
    )

File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/C02248B5/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/A02092E1/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01806B5/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01809A3/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B02009F6/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01806B6/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/B01809A4/segmentation.sh
File uploaded successfully to /home/ygong/ygong/AD/Stereo/Sample_integration/cell_segmentation/D02175A6/segmentation.sh
File uploaded successfully to /home/ygon

'/mnt/data_drive2/extra/ygong/Stereo/Stereo/Sample_integration/cell_segmentation'